In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score


In [4]:
df =pd.read_excel("df_limpio.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   folio               22735 non-null  int64         
 1   tag                 22735 non-null  object        
 2   folio_solicitud     22735 non-null  int64         
 3   fecha               22735 non-null  datetime64[ns]
 4   marca               22735 non-null  object        
 5   modelo              22735 non-null  object        
 6   plazo               22735 non-null  object        
 7   precio              22735 non-null  float64       
 8   enganche            22735 non-null  float64       
 9   descuento           22735 non-null  float64       
 10  semana              22735 non-null  int64         
 11  monto_financiado    22735 non-null  float64       
 12  costo_total         22735 non-null  int64         
 13  monto_accesorios    22735 non-null  float64   

**PRECIO**

Vamos a analizar la variable "precio" debido a que esta será nuestra varaible dependiente que pasará a convertirse en variable diatónica.
Gama baja - Gama media

In [5]:
precioStats = df["precio"].describe()
print(precioStats)

count    22735.000000
mean      3997.036591
std       1361.507164
min        973.240000
25%       3049.000000
50%       3549.000000
75%       4899.000000
max      14900.000000
Name: precio, dtype: float64


In [70]:
# Elegimos las variables de interés para poner a prueba el modelo
precio = df.iloc[ : , [8,9,11,12,13,25,26,27,29,30,7]]
precio.head(6
            )

,enganche,descuento,monto_financiado,costo_total,monto_accesorios,puntos,riesgo,score_buro,porc_eng,limite_credito,precio
0,780.0,0.0,1169.0,2106,0.0,0,0.0,0,40.02,5000,1949.0
1,1050.0,0.0,1949.0,2925,0.0,0,0.0,0,35.01,6000,2999.0
2,490.0,0.0,1469.0,2210,0.0,0,0.0,0,25.01,0,1959.0
3,490.0,0.0,1469.0,2210,0.0,0,0.0,0,25.01,7000,1959.0
4,1075.0,0.0,3224.0,6942,0.0,0,0.0,0,25.01,7000,4299.0
5,750.0,0.0,2249.0,4056,0.0,0,0.0,0,25.01,7000,2999.0


In [7]:

# Calcular la media de la columna
meanPrecio = precio["precio"].mean()

# Definir una función que asignará la categoría
def categorizar_valor(valor):
    if valor > meanPrecio:
        return 'Gama media'
    else:
        return 'Gama baja'

# Aplicar la función a la columna y crear una nueva columna con las categorías
precio['PrecioGama'] = precio["precio"].apply(categorizar_valor)

# Mostrar el DataFrame resultante
print(precio)


       enganche  descuento  monto_financiado  costo_total  monto_accesorios  \
0         780.0        0.0            1169.0         2106               0.0   
1        1050.0        0.0            1949.0         2925               0.0   
2         490.0        0.0            1469.0         2210               0.0   
3         490.0        0.0            1469.0         2210               0.0   
4        1075.0        0.0            3224.0         6942               0.0   
...         ...        ...               ...          ...               ...   
22730     297.0        0.0            3052.0         5486               0.0   
22731    1209.0        0.0            2891.0         5200               0.0   
22732    1152.0        0.0            3697.0         6656               0.0   
22733    2799.0        0.0            7000.0        15054               0.0   
22734    1086.0        0.0            4313.0         7774               0.0   

       puntos  riesgo  score_buro  porc_eng  limite

C:\Users\Carlos Casillas\AppData\Local\Temp\ipykernel_10104\2920784270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precio['PrecioGama'] = precio["precio"].apply(categorizar_valor)


In [8]:
#Verificamos que los campos no contengan valores nulos
precio.isnull().sum()

enganche            0
descuento           0
monto_financiado    0
costo_total         0
monto_accesorios    0
puntos              0
riesgo              0
score_buro          0
porc_eng            0
limite_credito      0
precio              0
PrecioGama          0
dtype: int64

In [9]:
#Declaramos variables dependientes e independientes
precio_indep = precio[["limite_credito","score_buro","puntos","enganche"]]
precio_dep = precio[["PrecioGama"]]

In [10]:
X1 = precio_indep
y1=precio_dep

In [11]:
#División del conjunto de datos
X1_train, X1_test, y1_train,y1_test =train_test_split(X1,y1,test_size =0.2, random_state= None)

In [12]:
escalarPrecio =StandardScaler()

In [13]:
X1_train =escalarPrecio.fit_transform(X1_train)
X1_test =escalarPrecio.transform(X1_test)    #por que uno es fit y el otro no

In [14]:
#Definimos el algoritmo
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [15]:
algoritmo.fit(X1_train,y1_train)

c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [16]:
#Se realiza la predicción
y1_pred =algoritmo.predict(X1_test)
y1_pred

array(['Gama media', 'Gama baja', 'Gama baja', ..., 'Gama media',
       'Gama baja', 'Gama baja'], dtype=object)

In [17]:
from sklearn.metrics import confusion_matrix
matriz1 = confusion_matrix(y1_test,y1_pred)
print(matriz1)

[[2469  176]
 [ 380 1522]]


In [18]:
from sklearn.metrics import precision_score

presicion1 =precision_score(y1_test, y1_pred, average = "binary", pos_label = "Gama baja")
print ("Precisión del modelo: ")
print(presicion1)

Precisión del modelo: 
0.8666198666198667


In [19]:
# Calculo la exactitud del modelo

from sklearn.metrics import accuracy_score

exactitud1 = accuracy_score(y1_test, y1_pred)
print('Exactitud del modelo: ')
print(exactitud1)

Exactitud del modelo: 
0.877721574664614


In [20]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad1 = recall_score(y1_test, y1_pred, average = "binary", pos_label = 'Gama baja')
print('Sensibilidad del modelo: ')
print(sensibilidad1)

Sensibilidad del modelo: 
0.9334593572778828


In [21]:
# El puntaje F1 es una combinación entre la precisión y la sensibilidad, para esto importamos
# Calculo el puntaje F1 del modelo

from sklearn.metrics import f1_score

puntajef1_precio = f1_score(y1_test, y1_pred, average = "binary", pos_label = "Gama baja")
print('Puntaje F1 del modelo: ')
print(puntajef1_precio)

Puntaje F1 del modelo: 
0.8987986894794321


In [22]:
class_report = classification_report(y1_test, y1_pred)
print('Classification Report:')
print(class_report)

Classification Report:
              precision    recall  f1-score   support

   Gama baja       0.87      0.93      0.90      2645
  Gama media       0.90      0.80      0.85      1902

    accuracy                           0.88      4547
   macro avg       0.88      0.87      0.87      4547
weighted avg       0.88      0.88      0.88      4547



**PUNTOS**

Dibidmos nuestra base de datos y utilizamos la mediana como discriminante ,este se verá categorizado por "Alto" y "bajo"

In [23]:
puntosStats = df["puntos"].describe()
print(puntosStats)

count    22735.000000
mean        14.156807
std         11.399142
min        -15.000000
25%         10.000000
50%         12.000000
75%         14.000000
max         50.000000
Name: puntos, dtype: float64


In [24]:
puntos = df.iloc[ : , [30,25]]

Dado que la naturaleza de esta variable no permite números negativos procederemos a eliminarlos.

In [25]:
puntos = puntos[(puntos["puntos"]>0)]
puntos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19475 entries, 928 to 22734
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   limite_credito  19475 non-null  int64
 1   puntos          19475 non-null  int64
dtypes: int64(2)
memory usage: 456.4 KB


In [26]:
# Calcular la media de la columna
medianPuntos = puntos["puntos"].median()

# Definir una función que asignará la categoría
def categorizar_valor(valor):
    if valor > medianPuntos:
        return 'Alto'
    else:
        return 'Bajo'

# Aplicar la función a la columna y crear una nueva columna con las categorías
puntos['CatPuntos'] = puntos["puntos"].apply(categorizar_valor)

# Mostrar el DataFrame resultante
print(puntos)

       limite_credito  puntos CatPuntos
928                 0      11      Bajo
1558             9000      31      Alto
1744                0      28      Alto
3186             9000      32      Alto
3187             9000      29      Alto
...               ...     ...       ...
22730           10000      10      Bajo
22731            5000       9      Bajo
22732            4750      11      Bajo
22733            7000      14      Alto
22734            4750      12      Bajo

[19475 rows x 3 columns]


In [27]:
#Verificamos nulos
puntos.isnull().sum()

limite_credito    0
puntos            0
CatPuntos         0
dtype: int64

In [28]:
puntos_indep = puntos[["limite_credito"]]
puntos_dep = puntos [["CatPuntos"]]

In [29]:
X2 =puntos_indep
y2 = puntos_dep

In [30]:
X2_train, X2_test,y2_train,y2_test = train_test_split(X2,y2,test_size = 0.2,random_state = None)

In [31]:
escalarPuntos = StandardScaler()

In [32]:
X2_train = escalarPrecio.fit_transform(X2_train)
X2_test = escalarPrecio.transform(X2_test)

In [33]:
algoritmo.fit(X2_train,y2_train)

c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [34]:
#Se realiza la predicción
y2_pred =algoritmo.predict(X2_test)
y2_pred

array(['Bajo', 'Bajo', 'Bajo', ..., 'Alto', 'Alto', 'Alto'], dtype=object)

In [35]:
from sklearn.metrics import confusion_matrix
matriz2 = confusion_matrix(y2_test,y2_pred)
print(matriz2)

[[1147  579]
 [ 295 1874]]


In [36]:
from sklearn.metrics import precision_score

presicion2 =precision_score(y2_test, y2_pred, average = "binary", pos_label = "Alto")
print ("Precisión del modelo: ")
print(presicion2)

Precisión del modelo: 
0.7954230235783634


In [37]:
from sklearn.metrics import accuracy_score

exactitud2 = accuracy_score(y2_test, y2_pred)
print('Exactitud del modelo: ')
print(exactitud2)

Exactitud del modelo: 
0.775609756097561


In [38]:
from sklearn.metrics import recall_score

sensibilidad2 = recall_score(y2_test, y2_pred, average = "binary", pos_label = 'Alto')
print('Sensibilidad del modelo: ')
print(sensibilidad2)

Sensibilidad del modelo: 
0.6645422943221321


In [39]:
from sklearn.metrics import f1_score

puntajef1_puntos = f1_score(y2_test, y2_pred, average = "binary", pos_label = "Alto")
print('Puntaje F1 del modelo: ')
print(puntajef1_puntos)

Puntaje F1 del modelo: 
0.7241161616161615


In [40]:
class_report = classification_report(y2_test, y2_pred)
print('Classification Report:')
print(class_report)

Classification Report:
              precision    recall  f1-score   support

        Alto       0.80      0.66      0.72      1726
        Bajo       0.76      0.86      0.81      2169

    accuracy                           0.78      3895
   macro avg       0.78      0.76      0.77      3895
weighted avg       0.78      0.78      0.77      3895



**StatusCuenta**

Esta variable es demasiado impornate para el análisis del cliente, mediante el análisis de estos que están al corriente y los que no lo están lograremos obtener las variables claves que nos indique a que posible grupo pueda categorizarse cada uno.

In [41]:
#1. Seleccionamos las variables de interés.
status = df.iloc[ : , [7,8,9,11,12,13,27,25,30,24]]

In [42]:
#2. Verificamos datos nulos
status.isnull().sum()

precio                 0
enganche               0
descuento              0
monto_financiado       0
costo_total            0
monto_accesorios       0
score_buro             0
puntos                 0
limite_credito         0
status_cuenta       3314
dtype: int64

In [43]:
# 3. Utilizaremos como variable dependinte "status_cuenta" con un filtro que catalogue "fraude" y "corriente"
status = status[(status["status_cuenta"]=="Fraude") | (status["status_cuenta"]== "Corriente")]
#status.info()

In [44]:
frecuenciaStatus = status["status_cuenta"].value_counts()
frecuenciaStatus

status_cuenta
Corriente    10014
Fraude        1696
Name: count, dtype: int64

In [45]:
status_indep = status[["precio","enganche","descuento","costo_total","monto_accesorios","score_buro","puntos","limite_credito"]]
status_dep = status[["status_cuenta"]]

In [46]:
X3 = status_indep
y3=status_dep

In [47]:
X3_train, X3_test, y3_train,y3_test =train_test_split(X3,y3,test_size =0.3, random_state= None)


In [48]:
escalarStatus = StandardScaler()

In [49]:
X3_train =escalarStatus.fit_transform(X3_train)
X3_test =escalarStatus.transform(X3_test)    

In [50]:
algoritmo.fit(X3_train,y3_train)

c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [51]:
y3_pred =algoritmo.predict(X3_test)
y3_pred

array(['Corriente', 'Corriente', 'Corriente', ..., 'Corriente',
       'Corriente', 'Corriente'], dtype=object)

In [52]:
from sklearn.metrics import confusion_matrix
matriz3 = confusion_matrix(y3_test,y3_pred)
print(matriz3)

[[2990    0]
 [ 521    2]]


In [53]:
precision3 =precision_score(y3_test, y3_pred, average = "binary", pos_label = "Corriente")
print ("Precisión del modelo: ")
print(precision3)

Precisión del modelo: 
0.8516092281401311


In [54]:
precision3 =precision_score(y3_test, y3_pred, average = "binary", pos_label = "Fraude")
print ("Precisión del modelo: ")
print(precision3)

Precisión del modelo: 
1.0


In [55]:
exactitud3 = accuracy_score(y3_test, y3_pred)
print('Exactitud del modelo: ')
print(exactitud3)

Exactitud del modelo: 
0.851693709080558


In [56]:
sensibilidad3 = recall_score(y3_test, y3_pred, average = "binary", pos_label = 'Corriente')
print('Sensibilidad del modelo: ')
print(sensibilidad3)


Sensibilidad del modelo: 
1.0


In [57]:
sensibilidad3 = recall_score(y3_test, y3_pred, average = "binary", pos_label = 'Fraude')
print('Sensibilidad del modelo: ')
print(sensibilidad3)

Sensibilidad del modelo: 
0.0038240917782026767


In [58]:
puntajef1_status = f1_score(y3_test, y3_pred, average = "binary", pos_label = "Corriente")
print('Puntaje F1 del modelo: ')
print(puntajef1_status)

Puntaje F1 del modelo: 
0.91985848331026


In [59]:
puntajef1_status = f1_score(y3_test, y3_pred, average = "binary", pos_label = "Fraude")
print('Puntaje F1 del modelo: ')
print(puntajef1_status)

Puntaje F1 del modelo: 
0.007619047619047619


**PORCENTAJE DE ENGANCHE**

Esta si es una variable interesante, parte del riesgo que se corre al terminar de pagar el equipo se basa en los plazos y la cantidad a pagar, por lo que esta variable nos explicará si necesitamos en enganche "neutro" o "alto".

In [60]:
#Exploración de la variable de interés
engStats = df["porc_eng"].describe()
print(engStats)

count    22735.000000
mean        23.189330
std          7.067519
min          0.000000
25%         18.450000
50%         22.900000
75%         25.800000
max         89.830000
Name: porc_eng, dtype: float64


In [61]:
enganche = df.iloc[ : , [27,29]]

Normlamnete se solicita un enganche del 25% al 30%, por lo que el método discriminante será el establecido por la empresa donde una persona que deja un enganche <30% será "normal" uno que necesite enganche >30% será "riesgo".

In [62]:
enganchePercent = 30

# Definir una función que asignará la categoría
def categorizar_valor(valor):
    if valor > enganchePercent:
        return "ClienteAR"
    else:
        return 'ClienteBR'

# Aplicar la función a la columna y crear una nueva columna con las categorías
enganche['porc_eng'] = enganche["porc_eng"].apply(categorizar_valor)

C:\Users\Carlos Casillas\AppData\Local\Temp\ipykernel_10104\257451076.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enganche['porc_eng'] = enganche["porc_eng"].apply(categorizar_valor)


In [63]:
X4 = enganche[["score_buro"]]
y4 = enganche[["porc_eng"]]

In [64]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2, random_state=42)


In [65]:
X4_train =escalarPrecio.fit_transform(X4_train)
X4_test =escalarPrecio.transform(X4_test)

In [66]:
logistic_model = LogisticRegression()
logistic_model.fit(X4_train, y4_train)

c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [67]:
y4_pred = logistic_model.predict(X4_test)


In [68]:
confusion = confusion_matrix(y4_test, y4_pred)
print('Confusion Matrix:')
print(confusion)

Confusion Matrix:
[[   0  537]
 [   0 4010]]


In [69]:
class_report = classification_report(y4_test, y4_pred)
print('Classification Report:')
print(class_report)

c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

   ClienteAR       0.00      0.00      0.00       537
   ClienteBR       0.88      1.00      0.94      4010

    accuracy                           0.88      4547
   macro avg       0.44      0.50      0.47      4547
weighted avg       0.78      0.88      0.83      4547



c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
